In [10]:
from bs4 import BeautifulSoup
import requests
import pickle
import imdb_funcs
import json

In [2]:
actor_file = open('popularActors.pkl', 'rb')

In [3]:
actor_list = pickle.load(actor_file)

In [4]:
actor_list[1] = 'Robert Downey'

In [5]:
actor_list[26] = 'Samuel Jackson'

In [6]:
actor_dict_list = imdb_funcs.get_actor_titles(actor_list)

In [7]:
tot = 0
for i in actor_dict_list:
    tot += len(i['titles'])

In [8]:
actor_dict_list

[{'actor_id': 'nm0262635',
  'name': 'Chris Evans',
  'url': 'https://www.imdb.com/name/nm0262635/',
  'titles': [('tt5697510', 'Jekyll'),
   ('tt8946378', 'Knives Out'),
   ('tt4995776', 'Operation Brothers'),
   ('tt4154796', 'Avengers: Endgame'),
   ('tt4154664', 'Captain Marvel'),
   ('tt4154756', 'Avengers: Infinity War'),
   ('tt2250912', 'Spider-Man: Homecoming'),
   ('tt4481414', 'Gifted'),
   ('tt3498820', 'Captain America: Civil War'),
   ('tt0478970', 'Ant-Man'),
   ('tt2395427', 'Avengers: Age of Ultron'),
   ('tt2139555', 'Playing It Cool'),
   ('tt0443465', 'Before We Go'),
   ('tt1843866', 'Captain America: The Winter Soldier'),
   ('tt1981115', 'Thor: The Dark World'),
   ('tt1706620', 'Snowpiercer'),
   ('tt1491044', 'The Iceman'),
   ('tt0848228', 'The Avengers'),
   ('tt0770703', "What's Your Number?"),
   ('tt1740721', 'Captain America: Super Soldier'),
   ('tt0458339', 'Captain America: The First Avenger'),
   ('tt1582248', 'Puncture'),
   ('tt0446029', 'Scott Pilg

In [11]:
with open('imdb_actors.json', 'w') as jfile:
    json.dump(actor_dict_list, jfile)

In [ ]:
sub_list = []
for item in actor_dict_list:
    for title in item['titles']:
        sub_list.append(title[0])

In [ ]:
sub_list

In [ ]:
movie_id_list = []
for i in actor_dict_list:
    sub_list = [title for title in i['titles']]
    movie_id_list.append()

In [ ]:
tot

In [ ]:
def imdb_movie_page_parser_v3(url):
    print(url)
    # request the url as defined in the url parameter above
    html_page = requests.get(url)
    # use beautiful soup to store content of website
    soup = BeautifulSoup(html_page.content, 'html.parser')
    #slice last 9 characters of URL string to get the movie ID
    movie_id = url[-9:]
    title = soup.findAll('div', {'class':'title_wrapper'})[0].find('h1').text.replace('\xa0', ' ').strip()
    director = soup.findAll('div', {'class':'credit_summary_item'})[0].text.replace('\n', '').strip().replace('Director:','')
    cast1 = soup.findAll('td')[2].text.replace('\n', '').strip()
    cast2 = soup.findAll('td')[6].text.replace('\n', '').strip()
    cast3 = soup.findAll('td')[10].text.replace('\n', '').strip()
    if len(soup.findAll('td')) > 14:
        cast4 = soup.findAll('td')[14].text.replace('\n', '').strip()
    else:
        cast4 = None
    # if there is not 5th or 6th cast member, the length of soup.findAll('td') list which contains all of the cast
    # is used to stop 'index out of range' errors
    if len(soup.findAll('td')) > 18:
        cast5 = soup.findAll('td')[18].text.replace('\n', '').strip()
    else:
        cast5 = None
    if len(soup.findAll('td')) > 22:
        cast6 = soup.findAll('td')[22].text.replace('\n', '').strip()
    else:
        cast6 = None
    
    if soup.findAll('div', {'class':'titleReviewBarItem'}):
        metascore = soup.findAll('div', {'class':'titleReviewBarItem'})[0].find('span').text
    else:
        metascore = None
    
    # the genres do not follow the same order within the findAll list initially parsed. As such the following is done
    genres_loc = soup.findAll('div', {'class':'see-more inline canwrap'}) #set the list container as a variable genres_loc
    x=0 #set x variable to zero to be used in the for loop to identify the location of the genres sub-container through its index number
    y=0 #set y variable to zero this will be used in the if statement below. Explaination of the statement to follow
    for i in range(len(genres_loc)): # i is the index number of the range-length of the genres_loc variable
        if str(genres_loc[i]).find('Genres:') > 0: # the keyword that always preceeds the genres sub-container is 'Genres:'
            x = i                                  #the .find() method returns a +ve number to indicate the location of the keyword
        else:                                      #if it does not find the keyword it will return -1 as output for the i-th container
            y-=1                                   #y variable add on the -1 values
    if y == len(genres_loc)*-1:                    #if the length of the main container, genres_loc, *-1 equals the y varialbe
        genres = None                              #the there is no specified genre and genres value is None
    else:                                          # else i will return the sub-container through the index number i and set it equal to the value of genres in the dictionary after trimming down the unwanted characters and spaces
        genres = soup.findAll('div', {'class':'see-more inline canwrap'})[x].text.strip().replace('\n','').replace('\xa0|',',').replace('Genres: ','').split(',')
        
    # for budget and box office numbers, the same problem is faced with the cast- thus same if statement is used
    if len(soup.findAll('div', {'class':'txt-block'})) > 9:
        budget = soup.findAll('div', {'class':'txt-block'})[9].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        budget = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 11:
        bo_gu = soup.findAll('div', {'class':'txt-block'})[11].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_gu = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 10:
        bo_op = soup.findAll('div', {'class':'txt-block'})[10].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_op = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 12:
        bo_c = soup.findAll('div', {'class':'txt-block'})[12].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_c = None
    
    studio_loc = soup.findAll('div', {'class':'txt-block'})
    k=0
    j=0
    for i in range(len(studio_loc)):
        if str(studio_loc[i]).find('Production Co:') > 0:
            j = i   
        else:
            k-=1
    if k == len(studio_loc)*-1:
        studio = None
    else:
        studio =  soup.findAll('div', {'class':'txt-block'})[j].next.next.next.next.next.next
    
    movie_dict={'movie_id':movie_id, 'title':title, 'director': director, 'cast': [cast1, cast2, cast3, cast4, cast5, cast6], 'genres':genres,
                   'metascore': metascore, 'budget': budget, 'box office gross usa': bo_gu, 
                'box office cumulative': bo_c, 'box office opening week': bo_op, 'studio': studio} #removed int() from all
    print(movie_dict)
    return movie_dict

In [ ]:
# def imdb_movie_page_parser_v3p(url):
#     # request the url as defined in the url parameter above
#     html_page = requests.get(url)
#     # use beautiful soup to store content of website
#     soup = BeautifulSoup(html_page.content, 'html.parser')
#     #slice last 9 characters of URL string to get the movie ID
#     movie_id = url[-9:]
#     title = soup.findAll('div', {'class':'title_wrapper'})[0].find('h1').text.replace('\xa0', ' ').strip()
#     director = soup.findAll('div', {'class':'credit_summary_item'})[0].text.replace('\n', '').strip().replace('Director:','')
#     cast1 = soup.findAll('td')[2].text.replace('\n', '').strip()
#     cast2 = soup.findAll('td')[6].text.replace('\n', '').strip()
#     cast3 = soup.findAll('td')[10].text.replace('\n', '').strip()
#     cast4 = soup.findAll('td')[14].text.replace('\n', '').strip()
#     # if there is not 5th or 6th cast member, the length of soup.findAll('td') list which contains all of the cast
#     # is used to stop 'index out of range' errors
#     if len(soup.findAll('td')) > 18:
#         cast5 = soup.findAll('td')[18].text.replace('\n', '').strip()
#     else:
#         cast5 = None
#     if len(soup.findAll('td')) > 22:
#         cast6 = soup.findAll('td')[22].text.replace('\n', '').strip()
#     else:
#         cast6 = None
#     metascore = soup.findAll('div', {'class':'titleReviewBarItem'})[0].find('span').text
    
#     # the genres do not follow the same order within the findAll list initially parsed. As such the following is done
#     genres_loc = soup.findAll('div', {'class':'see-more inline canwrap'}) #set the list container as a variable genres_loc
#     x=0 #set x variable to zero to be used in the for loop to identify the location of the genres sub-container through its index number
#     y=0 #set y variable to zero this will be used in the if statement below. Explaination of the statement to follow
#     for i in range(len(genres_loc)): # i is the index number of the range-length of the genres_loc variable
#         if str(genres_loc[i]).find('Genres:') > 0: # the keyword that always preceeds the genres sub-container is 'Genres:'
#             x = i                                  #the .find() method returns a +ve number to indicate the location of the keyword
#         else:                                      #if it does not find the keyword it will return -1 as output for the i-th container
#             y-=1                                   #y variable add on the -1 values
#     if y == len(genres_loc)*-1:                    #if the length of the main container, genres_loc, *-1 equals the y varialbe
#         genres = None                              #the there is no specified genre and genres value is None
#     else:                                          # else i will return the sub-container through the index number i and set it equal to the value of genres in the dictionary after trimming down the unwanted characters and spaces
#         genres = soup.findAll('div', {'class':'see-more inline canwrap'})[x].text.strip().replace('\n','').replace('\xa0|',',').replace('Genres: ','').split(',')
        
#     # for budget and box office numbers, the same problem is faced with the cast- thus same if statement is used
#     if len(soup.findAll('div', {'class':'txt-block'})) > 9:
#         budget = soup.findAll('div', {'class':'txt-block'})[9].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
#     else:
#         budget = None
#     if len(soup.findAll('div', {'class':'txt-block'})) > 11:
#         bo_gu = soup.findAll('div', {'class':'txt-block'})[11].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
#     else:
#         bo_gu = None
#     if len(soup.findAll('div', {'class':'txt-block'})) > 10:
#         bo_op = soup.findAll('div', {'class':'txt-block'})[10].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
#     else:
#         bo_op = None
#     if len(soup.findAll('div', {'class':'txt-block'})) > 12:
#         bo_c = soup.findAll('div', {'class':'txt-block'})[12].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
#     else:
#         bo_c = None
    
#     studio_loc = soup.findAll('div', {'class':'txt-block'})
#     k=0
#     j=0
#     for i in range(len(studio_loc)):
#         if str(studio_loc[i]).find('Production Co:') > 0:
#             j = i   
#         else:
#             k-=1
#     if k == len(studio_loc)*-1:
#         studio = None
#     else:
#         studio =  soup.findAll('div', {'class':'txt-block'})[j].next.next.next.next.next.next
    
#     movie_dict={'movie_id':movie_id, 'title':title, 'director': director, 'cast': [cast1, cast2, cast3, cast4, cast5, cast6], 'genres':genres,
#                    'metascore': metascore, 'budget': budget, 'box office gross usa': bo_gu, 
#                 'box office cumulative': bo_c, 'box office opening week': bo_op, 'studio': studio} #removed int() from all
#     data=movie_dict
#     pickles = []
#     with open('movies_sample.pkl', 'rb') as fr:
#         try:
#             while True:
#                 data.append(pickle.load(fr))
#         except EOFError:
#             pass
#     return movie_dict

# # data=LIST/DICT_YOU_WANT_TO_PICKLE
# # F=open('WhatYouWantToNameTheFile/SaveTheListAs.pkl','wb')
# # pickle.dump(data, F)
# # F.close()

In [ ]:
def imdb_movie_page_parser_v3_looper(list_of_movie_urls):
    movie_dic_list = []
    for element in list_of_movie_urls:
        movie_dic_list.append('https://www.imdb.com/title/'+element)
    return movie_dic_list

In [ ]:
url_list = imdb_movie_page_parser_v3_looper(sub_list)

In [ ]:
url_list

In [ ]:
final_list = []

In [ ]:
final_list[:50]

In [ ]:
for url in url_list:
    final_list.append(imdb_movie_page_parser_v3(url))

In [ ]:
import sys
sys.setrecursionlimit(100000)

In [ ]:
import json
with open('imdb_movies_50_actors.json', 'w') as jfile:
    json.dump(final_list, jfile)

In [ ]:
type(final_list)

In [ ]:
final_list[0]